In [ ]:
import torch
import torchvision
import torchvision.models as models
from torchvision.models import ResNet50_Weights
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import pandas as pd

# Встановлення директорії даних
data_dir = '/kaggle/input/intel-image-classification'

# Трансформації для тренувальних та тестових даних
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(150, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Завантаження даних
train_data = ImageFolder(root=f"{data_dir}/seg_train/seg_train", transform=train_transform)
test_data = ImageFolder(root=f"{data_dir}/seg_test/seg_test", transform=test_transform)

train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

class ModifiedResNet50(nn.Module):
    def __init__(self):
        super(ModifiedResNet50, self).__init__()
        self.base_model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_ftrs, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512, 6)

    def forward(self, x):
        x = self.base_model(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

model = ModifiedResNet50()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

results = []

def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_loss, val_acc, val_f1 = evaluate_model(model, val_loader)
        results.append({
            'epoch': epoch + 1,
            'train_loss': running_loss / len(train_loader),
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'val_f1_score': val_f1
        })
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}, Val Accuracy: {val_acc}%, Val F1-Score: {val_f1}")

def evaluate_model(model, data_loader):
    model.eval()
    total = 0
    correct = 0
    loss_total = 0
    all_targets = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss_total += loss.item()
            all_targets.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    accuracy = 100 * correct / total
    f1 = f1_score(all_targets, all_preds, average='macro')
    return loss_total / len(data_loader), accuracy, f1

train_model(model, criterion, optimizer, num_epochs=10)
evaluate_model(model, test_loader)

# Збереження результатів у CSV файл
df_results = pd.DataFrame(results)
df_results.to_csv('training_results.csv', index=False)
print("Results saved to 'training_results.csv'")

# Візуалізація матриці помилок
def plot_confusion_matrix(labels, predictions, class_names):
    cm = confusion_matrix(labels, predictions)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

_, predictions, labels = evaluate_model(model, test_loader)
plot_confusion_matrix(labels, predictions, train_data.classes)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 168MB/s] 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Завантаження результатів тренування з CSV
results_df = pd.read_csv('training_results.csv')

# Відображення перших кількох рядків даних для перевірки
print(results_df.head())

# Візуалізація динаміки зміни втрат і точності під час тренування
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(results_df['epoch'], results_df['train_loss'], label='Train Loss')
plt.plot(results_df['epoch'], results_df['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(results_df['epoch'], results_df['val_accuracy'], label='Validation Accuracy', color='green')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

def evaluate_and_visualize(model, loader, dataset_type, results_df):
    """ Оцінка моделі і візуалізація результатів для заданого набору даних. """
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')

    # Зберігання результатів у DataFrame
    new_row = pd.DataFrame({
        'Dataset Type': [dataset_type],
        'Accuracy': [accuracy],
        'F1 Score': [f1]
    })
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    print(f"{dataset_type} Accuracy: {accuracy:.2f}%")
    print(f"{dataset_type} F1-Score: {f1:.4f}")

    # Візуалізація матриці помилок
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=train_data.classes, yticklabels=train_data.classes)
    plt.title(f'Confusion Matrix for {dataset_type}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    return results_df


# Ініціалізація DataFrame для зберігання результатів
results_df = pd.DataFrame(columns=['Dataset Type', 'Accuracy', 'F1 Score'])

# Візуалізація та збереження для тренувального, валідаційного та тестового наборів
results_df = evaluate_and_visualize(model, train_loader, 'Training', results_df)
results_df = evaluate_and_visualize(model, val_loader, 'Validation', results_df)
results_df = evaluate_and_visualize(model, test_loader, 'Testing', results_df)

# Збереження результатів у CSV файл
results_df.to_csv('evaluation_results.csv', index=False)
print("Results saved to 'evaluation_results.csv'")


На основі представлених результатів, модель показує високу продуктивність на тренувальному наборі даних з точністю 99.53% та F1-балом 0.9954. Це свідчить про те, що модель добре адаптувалась до навчальних даних, можливо, навіть занадто добре, що може вказувати на перенавчання. 

На валідаційному наборі результати також досить високі з точністю 92.80% та F1-балом 0.9289. Це свідчить про те, що модель зберігає хорошу загальну продуктивність на даних, які вона раніше не бачила, але все ще є відмінність від тренувальних результатів.

Тестові результати є найкритичнішими Точність на тестовому наборі становить 93.83% із F1-балом 0.9396, що є відмінним показником і підтверджує, що модель ефективно узагальнює дані, які вона не бачила під час навчання.

Висновки:
Перенавчання: Є незначна ознака перенавчання, оскільки результати на тренувальному наборі вищі за валідаційний та тестовий набори. Це може бути вирішено за допомогою технік регуляризації,  застосування більш агресивної аугментації даних.

Загальна продуктивність: Модель показує дуже хорошу продуктивність на валідаційному та тестовому наборах, що вказує на її ефективність і потенціал для застосування в реальних сценаріях.

Можливості покращення: Для додаткового покращення моделі можна експериментувати з різними архітектурами, налаштуваннями гіперпараметрів або більш складними методами аугментації даних.

Перевірка узагальнення: Висока точність та F1-бал на тестовому наборі підтверджують, що модель добре узагальнює інформацію, що є ключовим аспектом при розробці ефективних моделей глибокого навчання.